In [1]:
"""
    savevar(fn, x)

saves the value of `x` to the file `fn`, where `fn` is the filename string of the file.
"""
savevar(fn, x) = write(fn, string(x))

"""
    loadvar(fn)

loads the file `fn` (the filename string of the file) and `Meta.parse |> eval`.
"""
loadvar(fn) = read(fn, String) |> Meta.parse |> eval

"""
    dir_savevar

is the default directory to which `@savevar` saves the values of variables.
"""
const dir_savevar = "tmp"

"""
    fn_savevar(x::Symbol)

is the filename string to which `@savevar` saves the value of a variable.
"""
fn_savevar(x::Symbol) = joinpath(dir_savevar, string(x) * ".txt")

"""
    @savevar(args...)

saves the variables in args to the corresponding textfiles.

Example: `@savevar A B C` saves the variables `A`, `B`, `C` to textfiles. 
"""
macro savevar(args...)
    A = [:(savevar($(fn_savevar(x)), $(esc(x)))) for x in args]
    quote $(A...); nothing end
end

"""
    @loadvar(args...)

loads the values from the textfiles corresponding to `args`.
If `length(args)` is greater than 1, then it returns the tuple of the values.

Example: `a, b, c = @loadvar A B C` loads 
the values of `A`, `B`, `C` in textfiles to the variables `a`, `b`, `c`.
"""
macro loadvar(args...)
    if length(args) == 1
        x = args[1]
        :(loadvar($(fn_savevar(x))))
    else
        A = [:(loadvar($(fn_savevar(x)))) for x in args]
        :(($(A...),))
    end
end

@loadvar

In [2]:
A = randn(ComplexF64, 4, 3, 2)

4×3×2 Array{ComplexF64, 3}:
[:, :, 1] =
  -0.15665-0.152185im  -0.00850843-0.117858im    0.991609+0.509163im
 -0.501556-0.395643im    -0.670763-0.303195im     1.10098+0.773024im
  0.816394-0.594176im   -0.0993448+0.869365im    -0.19295-0.0251947im
 -0.176706+0.434546im    -0.681203-0.354963im  -0.0460242+0.202243im

[:, :, 2] =
 -0.76602+0.280803im  -1.19998+1.13841im    0.415317+1.23584im
 -1.54378+0.581553im  -0.31096+0.219272im   -1.12813+0.258349im
 -0.38217+0.521425im   -1.5734+0.787018im  0.0892293-1.38347im
  0.84882-0.290673im  0.021213-0.556017im  -0.585268+1.49758im

In [3]:
@savevar A

In [4]:
read("tmp/A.txt", String)

"[-0.156650130215756 - 0.1521846045225801im -0.008508430031920991 - 0.11785832622730098im 0.9916085123898436 + 0.509163013160152im; -0.5015560579775442 - 0.39564252140474393im -0.6707627426813939 - 0.3031945722518282im 1.1009798923326821 + 0.7730243442095321im; 0.81639406" ⋯ 505 bytes ⋯ "4825026067im; -0.3821696633462519 + 0.5214246426767306im -1.5734002156414655 + 0.7870181178379331im 0.08922932667106047 - 1.3834685293567983im; 0.8488197258219703 - 0.2906733494342182im 0.02121301175192206 - 0.5560174926909549im -0.5852677181203029 + 1.49757517493166im]"

In [5]:
A_load = @loadvar A

4×3×2 Array{ComplexF64, 3}:
[:, :, 1] =
  -0.15665-0.152185im  -0.00850843-0.117858im    0.991609+0.509163im
 -0.501556-0.395643im    -0.670763-0.303195im     1.10098+0.773024im
  0.816394-0.594176im   -0.0993448+0.869365im    -0.19295-0.0251947im
 -0.176706+0.434546im    -0.681203-0.354963im  -0.0460242+0.202243im

[:, :, 2] =
 -0.76602+0.280803im  -1.19998+1.13841im    0.415317+1.23584im
 -1.54378+0.581553im  -0.31096+0.219272im   -1.12813+0.258349im
 -0.38217+0.521425im   -1.5734+0.787018im  0.0892293-1.38347im
  0.84882-0.290673im  0.021213-0.556017im  -0.585268+1.49758im

In [6]:
A_load == A

true

In [7]:
B = ["Foo", "Bar", "Baz"]

3-element Vector{String}:
 "Foo"
 "Bar"
 "Baz"

In [8]:
@savevar B

In [9]:
read("tmp/B.txt", String)

"[\"Foo\", \"Bar\", \"Baz\"]"

In [10]:
B_load = @loadvar B

3-element Vector{String}:
 "Foo"
 "Bar"
 "Baz"

In [11]:
B_load == B

true

In [12]:
D = Dict(:A => A, :B => B)

Dict{Symbol, Array} with 2 entries:
  :A => [-0.15665-0.152185im -0.00850843-0.117858im 0.991609+0.509163im; -0.501…
  :B => ["Foo", "Bar", "Baz"]

In [13]:
@savevar D

In [14]:
read("tmp/D.txt", String)

"Dict{Symbol, Array}(:A => [-0.156650130215756 - 0.1521846045225801im -0.008508430031920991 - 0.11785832622730098im 0.9916085123898436 + 0.509163013160152im; -0.5015560579775442 - 0.39564252140474393im -0.6707627426813939 - 0.3031945722518282im 1.1009798923326821 + 0.7730" ⋯ 561 bytes ⋯ "519 + 0.5214246426767306im -1.5734002156414655 + 0.7870181178379331im 0.08922932667106047 - 1.3834685293567983im; 0.8488197258219703 - 0.2906733494342182im 0.02121301175192206 - 0.5560174926909549im -0.5852677181203029 + 1.49757517493166im], :B => [\"Foo\", \"Bar\", \"Baz\"])"

In [15]:
D_load = @loadvar D

Dict{Symbol, Array} with 2 entries:
  :A => [-0.15665-0.152185im -0.00850843-0.117858im 0.991609+0.509163im; -0.501…
  :B => ["Foo", "Bar", "Baz"]

In [16]:
D_load == D

true

In [17]:
module O
struct Foo{A, B}
    a::A
    b::B
end
end

Main.O

In [18]:
foo = O.Foo(A, B)

Main.O.Foo{Array{ComplexF64, 3}, Vector{String}}([-0.156650130215756 - 0.1521846045225801im -0.008508430031920991 - 0.11785832622730098im 0.9916085123898436 + 0.509163013160152im; -0.5015560579775442 - 0.39564252140474393im -0.6707627426813939 - 0.3031945722518282im 1.1009798923326821 + 0.7730243442095321im; 0.8163940609702098 - 0.5941760728150809im -0.09934475697119184 + 0.8693647834487719im -0.19295033361398556 - 0.025194656996883568im; -0.17670641658760577 + 0.43454612004574245im -0.6812032779955188 - 0.35496302265308993im -0.0460242059900602 + 0.20224306598412567im;;; -0.7660201209718887 + 0.28080346633410824im -1.1999750338478339 + 1.13841192823082im 0.4153169662062253 + 1.2358439307770157im; -1.543778900140576 + 0.5815531889718983im -0.31095993516434856 + 0.21927228054858208im -1.1281253146424577 + 0.25834894825026067im; -0.3821696633462519 + 0.5214246426767306im -1.5734002156414655 + 0.7870181178379331im 0.08922932667106047 - 1.3834685293567983im; 0.8488197258219703 - 0.29067334

In [19]:
@savevar foo

In [20]:
read("tmp/foo.txt", String)

"Main.O.Foo{Array{ComplexF64, 3}, Vector{String}}([-0.156650130215756 - 0.1521846045225801im -0.008508430031920991 - 0.11785832622730098im 0.9916085123898436 + 0.509163013160152im; -0.5015560579775442 - 0.39564252140474393im -0.6707627426813939 - 0.3031945722518282im 1.10" ⋯ 578 bytes ⋯ "633462519 + 0.5214246426767306im -1.5734002156414655 + 0.7870181178379331im 0.08922932667106047 - 1.3834685293567983im; 0.8488197258219703 - 0.2906733494342182im 0.02121301175192206 - 0.5560174926909549im -0.5852677181203029 + 1.49757517493166im], [\"Foo\", \"Bar\", \"Baz\"])"

In [21]:
foo_load = @loadvar foo

Main.O.Foo{Array{ComplexF64, 3}, Vector{String}}([-0.156650130215756 - 0.1521846045225801im -0.008508430031920991 - 0.11785832622730098im 0.9916085123898436 + 0.509163013160152im; -0.5015560579775442 - 0.39564252140474393im -0.6707627426813939 - 0.3031945722518282im 1.1009798923326821 + 0.7730243442095321im; 0.8163940609702098 - 0.5941760728150809im -0.09934475697119184 + 0.8693647834487719im -0.19295033361398556 - 0.025194656996883568im; -0.17670641658760577 + 0.43454612004574245im -0.6812032779955188 - 0.35496302265308993im -0.0460242059900602 + 0.20224306598412567im;;; -0.7660201209718887 + 0.28080346633410824im -1.1999750338478339 + 1.13841192823082im 0.4153169662062253 + 1.2358439307770157im; -1.543778900140576 + 0.5815531889718983im -0.31095993516434856 + 0.21927228054858208im -1.1281253146424577 + 0.25834894825026067im; -0.3821696633462519 + 0.5214246426767306im -1.5734002156414655 + 0.7870181178379331im 0.08922932667106047 - 1.3834685293567983im; 0.8488197258219703 - 0.29067334

In [22]:
(foo_load.a, foo_load.b) == (foo.a, foo.b)

true

In [23]:
x, y, z = randn(3)
x, y, z

(-1.2309629892045488, -1.4670886856334742, 1.0166064458442237)

In [24]:
@savevar x y z

In [25]:
X, Y, Z = @loadvar x y z
X, Y, Z

(-1.2309629892045488, -1.4670886856334742, 1.0166064458442237)

In [26]:
(X, Y, Z) == (x, y, z)

true